In [1]:
!pip install pyspark

In [2]:
import pyspark as ps
from pyspark.sql import SparkSession

In [3]:
import findspark
findspark.init()

In [4]:
spark = SparkSession.builder.master("local[*]").appName("spark").getOrCreate()

In [5]:
sc = spark.sparkContext

In [6]:
dataset_parent_dir = 'G:/Big Data/'

In [7]:
league_min_df_path = dataset_parent_dir + 'matches-min.json'
league_df = spark.read.json(league_min_df_path)

In [8]:
league_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- gameCreation: long (nullable = true)
 |-- gameDuration: long (nullable = true)
 |-- gameId: long (nullable = true)
 |-- gameMode: string (nullable = true)
 |-- gameType: string (nullable = true)
 |-- gameVersion: string (nullable = true)
 |-- mapId: long (nullable = true)
 |-- participantIdentities: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- participantId: long (nullable = true)
 |    |    |-- player: struct (nullable = true)
 |    |    |    |-- accountId: string (nullable = true)
 |    |    |    |-- currentAccountId: string (nullable = true)
 |    |    |    |-- currentPlatformId: string (nullable = true)
 |    |    |    |-- matchHistoryUri: string (nullable = true)
 |    |    |    |-- platformId: string (nullable = true)
 |    |    |    |-- profileIcon: long (nullable = true)
 |    |    |    |-- summonerId: string (nullable = true)
 |    |    |    |-- summo

In [9]:
league_rdd = league_df.rdd

In [10]:
def get_league_name(row):
    item_lists = []
    for participant in row['participants']:
        stats = participant['stats']
        item_list = []
        for i in range(7):
            item = stats['item' + str(i)]
            item_name = item['name'] if item else None
            if item_name: item_list.append(item_name)
        item_lists.append((item_list, stats['win']))
    return item_lists

In [11]:
new_rdd = league_rdd.flatMap(get_league_name)

In [12]:
new_rdd.take(600)

[(['Shard of True Ice',
   "Athene's Unholy Grail",
   'Ardent Censer',
   "Mercury's Treads",
   'Cloth Armor',
   'Sapphire Crystal',
   'Oracle Lens'],
  False),
 (['Bilgewater Cutlass',
   'Iceborn Gauntlet',
   'Muramana',
   'Ninja Tabi',
   "Doran's Blade",
   'Dagger',
   'Farsight Alteration'],
  False),
 (['Enchantment: Cinderhulk',
   'Spirit Visage',
   "Giant's Belt",
   "Mercury's Treads",
   'Control Ward',
   'Oracle Lens'],
  False),
 (["Doran's Shield",
   'Hextech Protobelt-01',
   "Liandry's Torment",
   "Seeker's Armguard",
   "Mercury's Treads",
   'Warding Totem (Trinket)'],
  False),
 (['Corrupting Potion',
   "Zhonya's Hourglass",
   'Morellonomicon',
   'Rod of Ages',
   'Ninja Tabi',
   'Farsight Alteration'],
  False),
 (["Luden's Echo",
   'Morellonomicon',
   'Blasting Wand',
   "Seeker's Armguard",
   "Sorcerer's Shoes",
   'Broken Stopwatch',
   'Farsight Alteration'],
  True),
 (['Quicksilver Sash',
   'Blade of the Ruined King',
   "Mercury's Treads",
